In [ ]:
import gymnasium as gym
import os

import matplotlib.pyplot as plt
import numpy as np
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Double Deep Q 
class DQN(nn.Module):
    def __init__(self,learning_rate,input_dims,fc1_dims,fc2_dims,num_actions):
        super(DQN,self).__init__()
        
        print("input dimensions: ",input_dims[0],"number of actions: ",num_actions)
        
        # Three fully connected layers
        self.fc1 = nn.Linear(*input_dims,fc1_dims) # * unpacks the list of input_dims since it's 2-dimensional (same as input_dims[0])
        self.fc2 = nn.Linear(fc1_dims,fc2_dims)
        self.fc3 = nn.Linear(fc2_dims,num_actions)
        
        self.optimizer = optim.Adam(self.parameters(),lr=learning_rate)
        self.loss = nn.MSELoss()
        
        # Set device
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu:0')
        self.to(self.device)
        
    def forward(self,state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
# Agent 
class Agent:
    def __init__(self,discount,epsilon,learning_rate,input_dims,batch_size,num_actions,mem_size=500000,min_epsilon=0.05,epsilon_decay=0.0005):
        # Brain of the agent
        self.DQN = DQN(learning_rate,input_dims,256,256,num_actions)
        self.DQN_next = DQN(learning_rate,input_dims,256,256,num_actions)
        self.DQN_next.load_state_dict(self.DQN.state_dict()) # loads a model's parameter dictionary using a deserialized state_dict
        
        # Hyper-parameters
        self.discount = discount
        self.epsilon = epsilon
        self.min_epsilon = min_epsilon
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.action_space = [i for i in range(num_actions)]
        self.mem_size = mem_size
        self.batch_size = batch_size
        self.mem_counter = 0
        
        # Initialize memory 
        self.state_mem = np.zeros((mem_size,*input_dims),dtype=np.float32) # mem_size number of dim[0]*dim[1] matrices of zeros
        self.new_state_mem = np.zeros((mem_size,*input_dims),dtype=np.float32)
        self.action_mem = np.zeros(mem_size,dtype=np.int32)
        self.reward_mem = np.zeros(mem_size,dtype=np.float32)
        self.terminal_mem = np.zeros(mem_size,dtype=bool) # boolean value indicating whether it's the last memory
        
    # Store records inside the memory
    def storage(self,state,new_state,action,reward,terminal):
        index = self.mem_counter%self.mem_size # equal to mem_counter if mem_counter < mem_size
        self.state_mem[index] = state
        self.new_state_mem[index] = new_state
        self.reward_mem[index] = reward
        self.action_mem[index] = action
        self.terminal_mem[index] = terminal
        
        self.mem_counter += 1
        
    # Take actions
    def get_action(self,observation):
        if np.random.random() < self.epsilon:
            # Randomly choose one action from the action space
            action = np.random.choice(self.action_space)
        else:
            state = T.tensor([observation]).to(self.DQN.device)
            actions = self.DQN(state.float()) # same as self.to(torch.float32)
            action = T.argmax(actions).item() # the action corresponding to the index of the maximizing action in the actions tensor
        return action
    
    # Learning
    def learn(self):
        # Check memory counter against batch size since the latter comes from the former
        if self.mem_counter < self.batch_size:
            return
        
        # Initialize gradients
        self.DQN.optimizer.zero_grad()
        
        # Generate batch such that the batch size is smaller than the number of results in the memory
        max_batch = min(self.mem_counter,self.mem_size)
        batch = np.random.choice(max_batch,self.batch_size,replace=False)
        batch_index = np.arange(self.batch_size,dtype=np.int32)
        
        state_batch = T.tensor(self.state_mem[batch]).to(self.DQN.device)
        new_state_batch = T.tensor(self.new_state_mem[batch]).to(self.DQN.device)
        reward_batch = T.tensor(self.reward_mem[batch]).to(self.DQN.device)
        terminal_batch = T.tensor(self.terminal_mem[batch]).to(self.DQN.device)
        action_batch = self.action_mem[batch] # cannot follow the above format since used as index
        
        q_next = self.DQN_next(new_state_batch)
        q_eval = self.DQN(state_batch)[batch_index,action_batch]
        
        q_target = reward_batch+self.discount*T.max(q_next,dim=1)[0]
        q_target[terminal_batch] = 0.0
        
        # Backprop 
        loss = self.DQN.loss(q_target,q_eval).to(self.DQN.device)
        loss.backward() # get the gradient of the current tensor
        self.DQN.optimizer.step()
        self.update_epsilon()
        
    # Epsilon decay
    def update_epsilon(self):
        if self.epsilon > self.min_epsilon:
            self.epsilon = self.epsilon-self.epsilon_decay
    
    def update_network(self):
        self.DQN_next.load_state_dict(self.DQN.state_dict())      

        
# Model training
if __name__=='__main__':
    env = gym.make("LunarLander-v2", render_mode="rgb_array")
    input_dims = [env.observation_space.shape[0]]
    num_actions = env.action_space.n
    discount = 0.99
    epsilon = 1.0
    learning_rate = 0.001
    batch_size = 64
    
    agent = Agent(discount,epsilon,learning_rate,input_dims,batch_size,num_actions)
    
    scores = []
    avg_scores = []
    episodes = 500
    learn = True
    
    for i in range(episodes):
        score = 0
        state, _ = env.reset()
        done = False
        
        while not done:
            action = agent.get_action(state)
            new_state,reward,done,_,_ = env.step(action)
            
            if i%10 == 0:
                env.render()
            
            score += reward
            
            agent.storage(state,new_state,action,reward,done)
            if learn:
                agent.learn()
                # Update target every 5 episodes
                if i%5 == 0  and i != 0:
                    agent.update_network()
                    
            state = new_state
            
        scores.append(score)
        avg_score = np.mean(scores[-50:])
        avg_scores.append(avg_score)
        print("Episode: ",i,"\tScore: ",score,"\tAverage Score: %.3f"% avg_score,"Epsilon %.3f" % agent.epsilon)
        
        # Early stopping when average score of the last 10 episodes is above 200
        if np.mean(scores[-50:]) > 200:
            episodes = i+1
            break
    
    checkpoint = {"model":agent.DQN.state_dict(),"score":scores,"episodes":episodes} 
    #save_dir = "C:/Users/Yang Yue/OneDrive/Documents/GitHub/STAT_430/prep/"
    save_dir = "/Users/Yang/OneDrive/Documents/GitHub/STAT_430/prep/"
    T.save(checkpoint,save_dir+"lunarlander-DQN.pt")
    
    x = [i+1 for i in range(episodes)]
    plt.plot(x,scores,label="episode_reward")
    plt.plot(x,avg_scores,label="average_reward")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Learning Curve DQN")
    plt.legend()
    plt.show() 

input dimensions:  8 number of actions:  4
input dimensions:  8 number of actions:  4
Episode:  0 	Score:  -363.8836424586237 	Average Score: -363.884 Epsilon 0.992
Episode:  1 	Score:  -193.12331037097175 	Average Score: -278.503 Epsilon 0.959


/tmp/ipykernel_78813/3422936964.py:81: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1720538439675/work/torch/csrc/utils/tensor_new.cpp:278.)
  state = T.tensor([observation]).to(self.DQN.device)


Episode:  2 	Score:  -324.56298890098765 	Average Score: -293.857 Epsilon 0.901
Episode:  3 	Score:  -72.87556319854284 	Average Score: -238.611 Epsilon 0.829
Episode:  4 	Score:  -90.06653569081305 	Average Score: -208.902 Epsilon 0.795
Episode:  5 	Score:  -177.18244207438437 	Average Score: -203.616 Epsilon 0.739
Episode:  6 	Score:  -145.63593902882235 	Average Score: -195.333 Epsilon 0.686
Episode:  7 	Score:  -142.49795237802894 	Average Score: -188.729 Epsilon 0.632
Episode:  8 	Score:  -299.2153661786929 	Average Score: -201.005 Epsilon 0.577
Episode:  9 	Score:  -92.62250281041575 	Average Score: -190.167 Epsilon 0.491
Episode:  10 	Score:  -307.3231978759927 	Average Score: -200.817 Epsilon 0.441
Episode:  11 	Score:  -461.6839656964798 	Average Score: -222.556 Epsilon 0.050
Episode:  12 	Score:  -633.35093926589 	Average Score: -254.156 Epsilon 0.050
Episode:  13 	Score:  -533.2595301457407 	Average Score: -274.092 Epsilon 0.050
Episode:  14 	Score:  -498.3571606353604 	Aver